# 필로그(Feel-Log) — 데이터 EDA & 전처리 노트북

이 노트북은 **완전 초보자를 위한 셀 단위 가이드**입니다. 각 셀을 **위에서부터 차례대로** 실행하세요 (`Shift+Enter`).  
윈도우 경로가 이미 반영되어 있으며, 필요하면 첫 번째 셀에서 `BASE_DIR`만 바꾸면 됩니다.


## 03 — 라벨 JSON 기반 전처리 (bbox 크롭 + 224 규격 저장)

In [ ]:
# 버전 충돌 방지를 위해 호환 버전으로 설치 (한 번만 실행하면 됩니다)
!pip install -U "pip<25.3"
!pip install -U "numpy==1.26.4" \
               "pandas==2.2.2" \
               "matplotlib==3.8.4" \
               "pillow==10.3.0" \
               "scikit-learn==1.4.2" \
               "opencv-python==4.10.0.84"


In [ ]:
import os, json, glob
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

# 그래프 한글 폰트 이슈가 있으면 주석 해제하고 시스템에 맞는 폰트를 지정하세요.
# import matplotlib
# matplotlib.rc('font', family='Malgun Gothic')  # 윈도우 '맑은 고딕'
# plt.rcParams['axes.unicode_minus'] = False


In [ ]:
# Windows 한글 폰트 설정 (맑은 고딕)
from matplotlib import rcParams, font_manager
try:
    font_manager.fontManager.addfont(r"C:\Windows\Fonts\malgun.ttf")
    rcParams["font.family"] = "Malgun Gothic"
    rcParams["axes.unicode_minus"] = False
    print("Matplotlib font set to Malgun Gothic.")
except Exception as e:
    print("폰트 설정 실패:", e)


In [ ]:
\
# ★★★ 여기만 확인/수정하세요 ★★★
# 윈도우의 실제 데이터 폴더
BASE_DIR = Path(r"C:\Users\USER\Downloads\korean_emotion_complex_vision_1_percent")

# 서브 폴더들 (변경하지 않아도 됨)
LABELS_DIR = BASE_DIR / "labels"                # JSON들이 들어있는 폴더
EMOTION_DIR = BASE_DIR                           # 감정별 이미지 폴더들이 바로 아래에 있음 (중립/슬픔/상처/불안/분노/당황/기쁨)
PROCESSED_DIR = BASE_DIR / "processed"           # 전처리 결과 저장 폴더(자동 생성)
(PROCESSED_DIR / "from_json").mkdir(parents=True, exist_ok=True)
(PROCESSED_DIR / "from_folders").mkdir(parents=True, exist_ok=True)

BASE_DIR, LABELS_DIR, EMOTION_DIR, PROCESSED_DIR


### 1) 01번 노트북에서 만든 df 재생성 (또는 이 셀에서 직접 생성)

In [ ]:
import json, glob
def parse_json_to_rows(json_path: Path, default_label=None):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    rows = []
    if isinstance(data, dict) and 'annotations' in data and 'images' in data:
        imgs = {img['id']: img for img in data['images']}
        categories = {c['id']: c.get('name', c['id']) for c in data.get('categories', [])}
        for a in data['annotations']:
            img = imgs.get(a['image_id'], {})
            rows.append({
                'file_name': img.get('file_name'),
                'label': categories.get(a.get('category_id')) or default_label,
                'bbox': a.get('bbox')
            })
    elif isinstance(data, list):
        for e in data:
            rows.append({
                'file_name': e.get('image') or e.get('file_name') or e.get('filename'),
                'label': e.get('label') or e.get('emotion') or default_label,
                'bbox': e.get('bbox')
            })
    else:
        print(f"[경고] 지원하지 않는 구조: {json_path.name}")
    return rows

json_files = sorted(glob.glob(str(LABELS_DIR / "*_sampled.json")))
all_rows = []
for jp in json_files:
    default_label = Path(jp).stem.replace("_sampled","")
    all_rows.extend(parse_json_to_rows(Path(jp), default_label=default_label))
df = pd.DataFrame(all_rows)
df.head()


### 2) bbox 크롭 + 224×224 리사이즈 저장

In [ ]:
out_root = PROCESSED_DIR / "from_json"
out_root.mkdir(exist_ok=True)

processed = []

def safe_crop_box(im_w, im_h, x, y, w, h, margin=0.2):
    left = max(0, int(x - w*margin))
    top = max(0, int(y - h*margin))
    right = min(im_w, int(x + w + w*margin))
    bottom = min(im_h, int(y + h + h*margin))
    if right-left <= 0 or bottom-top <= 0:
        return None
    return (left, top, right, bottom)

for idx, r in df.iterrows():
    fname = r['file_name']
    label = (str(r['label']) if pd.notna(r['label']) else 'unknown').strip()
    in_path = (EMOTION_DIR / fname) if not Path(str(fname)).is_absolute() else Path(str(fname))
    if not in_path.exists():
        # 일부 JSON이 절대경로/상대경로 혼용일 수 있어 보조 탐색
        fallback = list(EMOTION_DIR.rglob(Path(str(fname)).name))
        if fallback:
            in_path = fallback[0]
        else:
            print("[누락]", fname)
            continue
    try:
        with Image.open(in_path) as im:
            # bbox가 있으면 크롭, 없으면 원본 사용
            b = r.get('bbox', None)
            if isinstance(b, (list, tuple)) and len(b) == 4:
                x,y,w,h = b
                box = safe_crop_box(im.width, im.height, x,y,w,h, margin=0.2)
                crop = im.crop(box) if box else im.copy()
            else:
                crop = im.copy()

            # 224 정방형 변환
            target = (224,224)
            crop = crop.convert('RGB').resize(target, Image.BICUBIC)

            # 저장
            out_dir = out_root / label
            out_dir.mkdir(parents=True, exist_ok=True)
            out_name = f"{in_path.stem}_{idx}.jpg"
            out_path = out_dir / out_name
            crop.save(out_path, quality=90)
            processed.append({'file': str(out_path), 'label': label})
    except Exception as e:
        print("처리 실패:", in_path, e)

df_proc = pd.DataFrame(processed)
df_proc.head(), len(df_proc)


### 3) train/val/test CSV 저장 (클래스 비율 유지 stratify)

In [ ]:
if len(df_proc) > 0 and df_proc['label'].nunique() > 1:
    train, temp = train_test_split(df_proc, test_size=0.3, stratify=df_proc['label'], random_state=42)
    val, test  = train_test_split(temp, test_size=0.5, stratify=temp['label'], random_state=42)
else:
    # 라벨이 하나뿐이면 stratify가 불가하므로 단순 분할
    m = int(len(df_proc)*0.7)
    t = int((len(df_proc)-m)/2)
    train, val, test = df_proc.iloc[:m], df_proc.iloc[m:m+t], df_proc.iloc[m+t:]

(train_path, val_path, test_path) = (BASE_DIR/"train.csv", BASE_DIR/"val.csv", BASE_DIR/"test.csv")
df_proc.to_csv(BASE_DIR/"processed_labels.csv", index=False)
train.to_csv(train_path, index=False)
val.to_csv(val_path, index=False)
test.to_csv(test_path, index=False)

train_path, val_path, test_path
